# Imports & Settings

In [1]:
# Core tools
import joblib
import pandas as pd

# Company info
import yfinance as yf

# Dash
from dash.dependencies import Input, Output

# Explainer Dashboard
from explainerdashboard.custom import *
from explainerdashboard import RegressionExplainer, ExplainerDashboard

# Setting Ticker
Until functionality is built directly into the dashboard for switching between companies, the ticker to be analyzed will need to be manually specified here.

In [2]:
ticker = 'MSFT'

# Loading Data
## S&P 500

In [3]:
snp = pd.read_csv('data/sp500.csv')
snp.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


## Company Info

In [4]:
info = yf.Ticker(ticker).info
info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'f

In [5]:
company = snp[snp.Symbol == ticker].Security.values[0]
sector = snp[snp.Symbol == ticker]['GICS Sector'].values[0]
industry = snp[snp.Symbol == ticker]['GICS Sub-Industry'].values[0]
hq = snp[snp.Symbol == ticker]['Headquarters Location'].values[0]
founded = snp[snp.Symbol == ticker].Founded.values[0]
marketcap = f"${info['marketCap']:,}"
website = info['website']
description = info['longBusinessSummary']
logo = info['logo_url']

## Price History

In [6]:
prices = pd.read_csv(f'data/price_histories/{ticker}_history.csv', index_col='Date')
prices.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061491,1.031789e+09
1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063687,3.081600e+08
1986-03-17,0.100694,0.103299,0.100694,0.102431,0.064785,1.331712e+08
1986-03-18,0.102431,0.103299,0.098958,0.099826,0.063138,6.776640e+07
1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062040,4.789440e+07


## Prebuilt Model

In [7]:
data_full = pd.read_csv(f'data/preprocessed_data/{ticker}_preprocessed.csv')
data = data_full.drop(columns=['Report Date', 'Price Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume'])
data.columns = [col.replace('.', '') for col in data.columns]
X = data.drop(columns=['Close'])
y = data.Close

In [8]:
model = joblib.load(f'models/{ticker}_model.joblib')
type(model)

xgboost.sklearn.XGBRFRegressor

In [9]:
model

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.2,
               colsample_bytree=0.2, gamma=50, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.5, max_delta_step=0, max_depth=4,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', random_state=42, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

# Setting Explainer

In [10]:
explainer = RegressionExplainer(model, X, y, shap='tree')

Generating self.shap_explainer = shap.TreeExplainer(model)


# Customizing Layout

## Overview Tab

In [11]:
class OverviewTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='Overview')
        
    def layout(self):
        return dbc.Container(
            children=[
                dbc.Row([
                    dbc.Col([
                        html.Div([
                            html.Img(
                                src=logo, 
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem'}
                            ),
                            html.H1(
                                children=html.B(f'{company} ({ticker})'),
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem',
                                       'marginLeft': '20px'}
                            )
                        ]),
                        html.Hr(),
                        html.Table(
                            children=[
                                html.Tr([
                                    html.Td(html.B('Sector:')),
                                    html.Td(sector),
                                    html.Td(html.B('Headquarters:')),
                                    html.Td(hq),
                                    html.Td(html.B('Market Cap:')),
                                    html.Td(marketcap)
                                ]),
                                html.Tr([
                                    html.Td(html.B('Industry:')),
                                    html.Td(industry),
                                    html.Td(html.B('Founded:')),
                                    html.Td(founded),
                                    html.Td(html.B('Website:')),
                                    html.Td(html.A(website, href=website, target='_blank'))
                                ])
                            ], 
                            style={'border-collapse': 'separate', 
                                   'border-spacing': '20px 5px',
                                   'margin-left': '-20px'}
                        ),
                        html.Br()
                    ])
                ]),
                dbc.Row([
                    dbc.Col([
                        html.H4(html.B('Description')),
                        html.P(description),
                        html.Br()
                    ])
                ]),
                dbc.Row([
                    dbc.Col([
                        html.H4(html.B('Price History')),
                        dcc.Graph(
                            id='price_history_graph',
                            figure={
                                'data': [{
                                    'x': prices.index, 
                                    'y': prices.Close, 
                                    'type': 'line'
                                }],
                                'layout': {
                                    'margin': dict(t=10),
                                    'xaxis': dict(title='Year'),
                                    'yaxis': dict(title='Price per Share',
                                                  tickprefix='$')
                                }
                            },
                            config={'displayModeBar': False}
                        )
                    ])
                ])
            ],
            style={'marginTop': '25px'}
        )

## Features Tab

In [12]:
class FeaturesTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='Features')
        
    def component_callbacks(self, app, **kwargs):
        @app.callback(
            Output('feature_graph', 'figure'),
            Input('feature_explorer_dropdown', 'value')
        )
        def update_graph(value):
            return {
                'data': [{
                    'x': data_full['Report Date'],
                    'y': data_full[value],
                    'type': 'bar'
                }],
                'layout': {
                    'margin': dict(t=25),
                    'xaxis': dict(title='Year'),
                    'yaxis': dict(title='Value (USD)',
                                  tickprefix='$')
                }
            }
    
    def layout(self):
        return dbc.Container(
            children=[
                dbc.Row([
                    dbc.Col([
                        html.Div([
                            html.Img(
                                src=logo, 
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem'}
                            ),
                            html.H1(
                                children=html.B(f'{company} ({ticker})'),
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem',
                                       'marginLeft': '20px'}
                            )
                        ]),
                        html.Hr(),
                    ])
                ]),
                dbc.Row([
                    dbc.Col([html.H4(html.B('Feature Explorer'))]),
                    dbc.Col([
                        dcc.Dropdown(id='feature_explorer_dropdown', 
                                     options=[{'label': col, 'value': col} for col in X.columns],
                                     value='Total Assets',
                                     multi=False)
                    ])
                ]),
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(id='feature_graph', 
                                  config={'displayModeBar': False})
                    ])
                ])
            ],
            style={'marginTop': '25px'}
        )

## SHAP Tab

In [13]:
class SHAPTab(ExplainerComponent):
    def __init__(self, explainer, name=None, **kwargs):
        super().__init__(explainer, title='SHAP Analysis')
        
        self.feat_imps = ImportancesComponent(
                             explainer,
                             depth=15,
                             no_permutations=True,
                             hide_popout=True
                         )
        
        self.depend = ShapDependenceComponent(
                             explainer, 
                             hide_popout=True,
                             hide_outliers=True,
                             hide_footer=True,
                             hide_index=True
                         )
        
        self.interaction = InteractionSummaryComponent(
                             explainer,
                             hide_popout=True,
                             hide_type=True,
                             depth=10
                         )
        
        self.inter_plot = InteractionDependenceComponent(
                             explainer,
                             hide_popout=True,
                             hide_index=True,
                         )
    
    def layout(self):
        return dbc.Container(
            children=[
                dbc.Row([
                    dbc.Col([
                        html.Div([
                            html.Img(
                                src=logo, 
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem'}
                            ),
                            html.H1(
                                children=html.B(f'{company} ({ticker})'),
                                style={'display': 'inline',
                                       'vertical-align': 'middle',
                                       'height': '3rem',
                                       'width': '3rem',
                                       'marginLeft': '20px'}
                            )
                        ]),
                        html.Hr(),
                    ])
                ]),
                dbc.Row([dbc.Col([self.feat_imps.layout()])]),
                html.Br(),
                dbc.Row([dbc.Col([self.depend.layout()])]),
                html.Br(),
                dbc.Row([dbc.Col([self.interaction.layout()])]),
                html.Br(),
                dbc.Row([dbc.Col([self.inter_plot.layout()])]),
                html.Br()
            ],
            style={'marginTop': '25px'}
        )

# Running the Dashboard

In [15]:
db = ExplainerDashboard(explainer,
                        tabs=[OverviewTab, FeaturesTab, SHAPTab],
                        title='Stock KPIs',
                        description='',
                        bootstrap=dbc.themes.SANDSTONE,
                        fluid=False,
                        header_hide_download=True,
                        hide_poweredby=True)

db.run(port=8050)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://74.129.178.98:8050
Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app 'explainerdashboard.dashboards' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://74.129.178.98:8050/ (Press CTRL+C to quit)
